# TensorFlow Notes

This page documents some simple notes to do with using TensorFlow. The full descriptions of some of the contained code examples can be found on the TensorFlow website,

The most basic example of a tensorflow application is the following HelloWorld, which I will explain :3

In [1]:
import tensorflow as tf 

# Simply print Hello, World!
hello = tf.constant('Hello, World!')
session = tf.Session()
print(session.run(hello))

# Run a simple math computation
a = tf.constant(10)
b = tf.constant(10)
print(session.run(a + b))

b'Hello, World!'
20


This application simply creates some constants and passes those commands to the TensorFlow session to compute, returning the result from the Session.run() function.

## MNIST Beginner Application

The following example does some simple hand-written digit analysis on tensorflow's example dataset. 

In [14]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# mnist now consists of three training sets:
# - train      # for training data 
# - test       # for testing 
# - validation # for validating results 
# 
# mnist.train.images contains all the images for the computation 
# mnist.train.labels contains labels for stuff? 

# A placeholder is a value we expect tensorflow to calculate 
x = tf.placeholder(tf.float32, [None, 784])

# Next we instantiate some variables for sending through tf 
# Here, w is the weights and b is the biases 
w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Now calculate our model using softmax regression. This is the 
# neural network formula, basically, which is just all the weights
# of the network multiplied by the inputs, plus the biases. 
y = tf.nn.softmax(tf.matmul(x, w) + b)

# Create another placeholder for storing the correct answers. 
y_ = tf.placeholder(tf.float32, [None, 10])

# Now we calculate the cross-entropy! Which is the calculation of 
# the correctness of the model. 
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

# Now we calculate the actual backpropogation algorithm. 
# Here 0.01 is the training rate, and minimize is the 
# function meant to calculate the final value. 
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Now we initialize the variables for our model. 
init = tf.initialize_all_variables()

# Create a new session. We interact with tensorflow through
# session objects that store the abstract syntax tree created
# up to this point. Session.run(...) is used to execute a 
# constructed command. 
session = tf.Session()
session.run(init)

# Now we run our training step 1000 times! 
for i in range(1000):
    # Grab next 100 elements 
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # Run the training step 
    session.run(train_step, feed_dict={x: batch_xs, y_: batch_ys}) 
    

# Now we match the predicted values against the actual values 
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

# Now we determine what number of predictions were correct 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Print the accuracy of the test data 
print(session.run(
        accuracy, 
        feed_dict={
            x: mnist.test.images, 
            y_: mnist.test.labels
        }))

# ACCURACY OF 91% YO! 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.9105


## Advanced MNIST Example

Now we move on to the more complicated material; using a deep convolutional neural network to classify digits from the above dataset.

First we start by loading the data:

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Success


Next we want to import tensorflow and start a new interactive session.

In [ ]:
import tensorflow as tf 
session = tf.InteractiveSession() 

This example first starts by creating a simple single-layer network using a softmax regression, then extends this model into a full multi-layer convolutional network. First we need to start by Creating some placeholder values:

In [ ]:
x  = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

Here, x denotes the input images from the MNIST dataset; each image flattens out into a 784-dimensional vector, so the dimension [None, 784] denotes a 1-dimensional vector. y_ denotes the output values for the given x vector image, which we will use to compare against our trained network.

Now we create our weights and biases for the network, as with the above example.

In [4]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

NameError: name 'tf' is not defined

Since the resultant network is only going to be one-dimensional, we simply have one layer casting the dimensions of the input to teh dimensions of the output.

Next we need to initialize the variables used in our model:

In [ ]:
session.run(tf.initialize_all_variables())

This function will initialize all of the Variable objects used in our computation before we begin. 

Finally, we Create our network by multiplying all of the weights together.

In [ ]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

Then calculate the cross entropy of the model:

In [ ]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

Now again we do the training, using a gradient descent optimizer with a weight of 0.01:

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

for i in range(1000):
    batch = mnist.next_batch(50)
    train_step.run(feed_dict={
        x: batch[0],
        y_: batch[1]
    })

Next we can get some heuristics on how well the model did:

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cas(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_: mnist.test.labels
        }))

Now let's build a multi-layer convolutional neural network! Yay :D

First we start by initializing our weights and biases:

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    intial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

These functions will take some given shape and create the corresponding matrices for representing the given data. 

The next step is to create some functions that will represent our convolution and pooling layers. This will be using max-pooling over 2x2 input blocks.

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(
        x, 
        W, 
        strides=[1, 1, 1, 1], 
        padding='SAME'
    )

def max_pool_2x2(x):
    return tf.nn.max_pool(
        x,
        ksize=[1,2,2,1],
        strides=[1,2,2,1],
        padding='SAME'
    )

Now we will initialize our first layers using these parameterized functions. The convolutional layer will compute 32 "features" for 5x5 filters from the previous layer. This is represented by the array [5, 5, 1, 32]. 

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

Now we need to reshape the input image into a 4-dimensional tensor. Here the dimensions are represented as such: [-1, 28, 28, 1], where the second and third dimensions represent the sizes of the images, and the final dimension represents the color channel (1, in this instance, since there are no RGB values).

In [ ]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

Now we feed the image into the input convolutional layer of the network and feed it into the ReLU (Rectified Linear Unit) function. 

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Now we create some fully-connected layers to finish the network.

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Now we add a dropout layer for preventing overfitting.

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

And finally we add a softmax layer for calculating the final output.

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

Now we get some heuristics from the model and report some statistics:

In [ ]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
session.run(tf.initialize_all_variables())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0],
            y_: batch[1],
            keep_prob: 1.0
        })
        
        print("step %d, training accuracy %g" % (i, train_accuracy))
    
    train_step.run(feed_dict={
        x: batch[0], 
        y_ batch[1],
        keep_prob: 0.5
    })
    
print("Test accuracy %g" % accuracy.eval(feed_dict={
    x: mnist.test.images,
    y_: mnist.test.labels, 
    keep_prob: 1.0 
}))

Now to bring everything together into one final product. 

In [2]:
# Retrieve MNIST Data 
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Create our tensorflow session
import tensorflow as tf 
session = tf.InteractiveSession() 

x  = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# W = tf.Variable(tf.zeros([784, 10]))
# b = tf.Variable(tf.zeros([10]))

# session.run(tf.initialize_all_variables())

# y = tf.nn.softmax(tf.matmul(x, W) + b)
# cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# for i in range(1000):
#     batch = mnist.train.next_batch(50)
#     train_step.run(feed_dict={
#         x: batch[0],
#         y_: batch[1]
#     })
    
# correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# print(accuracy.eval(feed_dict={
#     x: mnist.test.images,
#     y_: mnist.test.labels
# }))


# Utility functions for creating network layers 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial) 

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')


# Some network layers 
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)



W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)



W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1,7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)



keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
session.run(tf.initialize_all_variables())

for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0],
            y_: batch[1],
            keep_prob: 0.5
        })
        
        print("Step %d, training accuracy %g" % (i, train_accuracy))
        
    train_step.run(feed_dict={
        x: batch[0],
        y_: batch[1],
        keep_prob: 0.5
    })
    
print("Test accuracy %g" % accuracy.eval(feed_dict={
    x: mnist.test.images,
    y_: mnist.test.labels,
    keep_prob: 1.0
}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Step 0, training accuracy 0.16
Step 100, training accuracy 0.64
Step 200, training accuracy 0.76
Step 300, training accuracy 0.72
Step 400, training accuracy 0.88
Step 500, training accuracy 0.92
Step 600, training accuracy 0.98
Step 700, training accuracy 0.94
Step 800, training accuracy 0.9
Step 900, training accuracy 0.94
Step 1000, training accuracy 0.92
Step 1100, training accuracy 0.9
Step 1200, training accuracy 0.98
Step 1300, training accuracy 0.94
Step 1400, training accuracy 0.92
Step 1500, training accuracy 0.98
Step 1600, training accuracy 0.9
Step 1700, training accuracy 0.98
Step 1800, training accuracy 0.96
Step 1900, training accuracy 0.98
Test accuracy 0.9787
